# Rollingsales ETL

In [1]:
import pandas as pd
import re

from os import environ
from geopy import GoogleV3

Read the excel file with house sales data.

In [2]:
df_bronx = pd.read_excel('../../datasets/csv_datasets/rollingsales_bronx.xlsx', skiprows=4)
df_brooklyn = pd.read_excel('../../datasets/csv_datasets/rollingsales_brooklyn.xlsx', skiprows=4)
df_manhattan = pd.read_excel('../../datasets/csv_datasets/rollingsales_manhattan.xlsx', skiprows=4)
df_queens = pd.read_excel('../../datasets/csv_datasets/rollingsales_queens.xlsx', skiprows=4)
df_statenisland = pd.read_excel('../../datasets/csv_datasets/rollingsales_statenisland.xlsx', skiprows=4)

## Preparing to geocoding

Concatenate the dataframes that represent single boroughs.

In [3]:
df = pd.concat(objs=[df_bronx, df_brooklyn, df_manhattan, df_queens, df_statenisland], axis='index')

Reset the index, providing a new auto increment one.

In [4]:
df = df.reset_index(drop=True)

Map the borough code, into its name.

In [5]:
df['BOROUGH'] = df['BOROUGH'].map({
    1: 'MANHATTAN', 
    2: 'BRONX',
    3: 'BROOKLYN', 
    4: 'QUEENS', 
    5: 'STATEN ISLAND'
})

In [6]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,4453 PARK AVENUE,NaN,...,1.0,0.0,1.0,1646.0,1497.0,1899.0,1,A1,215000,2023-04-18
1,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,4453 PARK AVENUE,NaN,...,1.0,0.0,1.0,1646.0,1497.0,1899.0,1,A1,570000,2023-08-23
2,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3035,52,NaN,A1,461 EAST 178 STREET,NaN,...,1.0,0.0,1.0,1782.0,1548.0,1899.0,1,A1,0,2023-04-14
3,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3053,86,NaN,S0,2364 WASHINGTON AVENUE,NaN,...,1.0,2.0,3.0,1911.0,4080.0,1931.0,1,S0,0,2023-10-24
4,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2904,22,NaN,B9,454 EAST 172 STREET,NaN,...,2.0,0.0,2.0,1658.0,1428.0,1901.0,1,B9,350000,2023-06-26


In [7]:
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
73446,STATEN ISLAND,WOODROW,05 TAX CLASS 1 VACANT LAND,1,6970,84,NaN,A1,N/A LENEVAR AVENUE,NaN,...,1.0,0.0,1.0,2603.0,0.0,2022.0,1,V0,1116638,2022-12-19
73447,STATEN ISLAND,WOODROW,05 TAX CLASS 1 VACANT LAND,1,6970,85,NaN,A1,N/A LENEVAR AVENUE,NaN,...,1.0,0.0,1.0,2551.0,0.0,NaN,1,V0,1116638,2022-12-19
73448,STATEN ISLAND,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,6970,86,NaN,V0,N/A LENEVAR AVENUE,NaN,...,0.0,0.0,0.0,2597.0,0.0,2023.0,1,V0,1116638,2022-12-19
73449,STATEN ISLAND,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,6970,87,NaN,V0,96 LENEVAR AVENUE,NaN,...,0.0,0.0,0.0,2644.0,0.0,2023.0,1,V0,1116638,2022-12-19
73450,STATEN ISLAND,WOODROW,22 STORE BUILDINGS,4,7144,510,NaN,K2,3120 ARTHUR KILL ROAD,NaN,...,0.0,1.0,1.0,98116.0,7795.0,1977.0,4,K2,5600000,2022-12-19


Drop the rows where the number of the house is missing. It is possible to geocode these addresses.

In [8]:
df = df.drop(
    index=df[df['ADDRESS'].str.startswith('N/A')].index
)

Retrieve addresses where the street number is not specified correctly.

In [9]:
# Wrong street format
df_wsf_mask = df['ADDRESS'].str.match(r'\d{1,4}\-*\d{0,4}\s\d*[A-Z\s]*\d{1,4}\s(STREET|AVENUE)')
df[df_wsf_mask]

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
2,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3035,52,NaN,A1,461 EAST 178 STREET,NaN,...,1.0,0.0,1.0,1782.0,1548.0,1899.0,1,A1,0,2023-04-14
4,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2904,22,NaN,B9,454 EAST 172 STREET,NaN,...,2.0,0.0,2.0,1658.0,1428.0,1901.0,1,B9,350000,2023-06-26
7,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2912,144,NaN,B1,3813 3 AVENUE,NaN,...,2.0,0.0,2.0,2000.0,2400.0,1993.0,1,B1,763000,2023-09-07
8,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2912,151,NaN,B1,519 EAST 171 STREET,NaN,...,2.0,0.0,2.0,2000.0,2400.0,1993.0,1,B1,750000,2023-05-18
10,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2929,114,NaN,B1,3854 3 AVENUE,NaN,...,2.0,0.0,2.0,2883.0,2394.0,1995.0,1,B1,815000,2023-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70265,STATEN ISLAND,NEW DORP,01 ONE FAMILY DWELLINGS,1,4242,24,NaN,A2,282 8 STREET,NaN,...,1.0,0.0,1.0,4600.0,1992.0,1965.0,1,A2,730000,2023-09-29
70291,STATEN ISLAND,NEW DORP,02 TWO FAMILY DWELLINGS,1,4197,4,NaN,B9,197 3 STREET,NaN,...,2.0,0.0,2.0,3000.0,1800.0,2003.0,1,B9,0,2023-08-01
70292,STATEN ISLAND,NEW DORP,02 TWO FAMILY DWELLINGS,1,4210,22,NaN,B3,36 8 STREET,NaN,...,2.0,0.0,2.0,5500.0,2110.0,1930.0,1,B3,0,2023-04-11
70293,STATEN ISLAND,NEW DORP,02 TWO FAMILY DWELLINGS,1,4213,26,NaN,B2,70 8 STREET,NaN,...,2.0,0.0,2.0,5000.0,3066.0,1901.0,1,B2,906526,2023-03-03


In [10]:
def adjust_street_format(x: str):
    fap = re.split(pattern=r'\s(STREET|AVENUE)', string=x)[0] # first part of the address

    suffix_dict = {'1': 'ST', '2': 'ND', '3': 'RD'}

    street_number = re.split(pattern='\s', string=fap)[-1]

    # Obtains the correct suffix to concatenate
    th_condition = (
        # Conditions on last number of the street
        int(street_number[-1]) >= 4 or 
        int(street_number[-1]) == 0 or
        # Conditions if the number end with a number between 11 and 19
        (len(street_number) >= 2 and street_number[-2] == '1')
    )
    
    # Selects the suffix to apply
    suffix = ('TH' if th_condition else suffix_dict[street_number[-1]])

    fap += suffix    

    return fap + (' STREET' if 'STREET' in x else ' AVENUE')

Adjust the address format, using `map` with the previously defined function.

In [11]:
df.loc[df_wsf_mask, 'ADDRESS'] = df.loc[df_wsf_mask, 'ADDRESS'].map(adjust_street_format)

Remove the appartement's number since it's not useful when geocoding the address.

In [12]:
# A mask used to retrieve the appartement whose address contains an appartement's number
df_an_mask = df['ADDRESS'].str.contains(r',{1}')

In [13]:
# Remove the appartement's number from the address
df.loc[df_an_mask, 'ADDRESS'] = df[df_an_mask].loc[:, 'ADDRESS'].str.split(',').str[0]

In order to have a complete address, that can be geocoded, we need two information that will be joined for every feature value inside ADDRESS:

- Borough name (found in the BOROUGH column)
- City (which is New York City [NYC])

In [14]:
df['ADDRESS'] = df['ADDRESS'] + ', ' + 'NYC, ' + df['BOROUGH']

In [15]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,"4453 PARK AVENUE, NYC, BRONX",NaN,...,1.0,0.0,1.0,1646.0,1497.0,1899.0,1,A1,215000,2023-04-18
1,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,66,NaN,A1,"4453 PARK AVENUE, NYC, BRONX",NaN,...,1.0,0.0,1.0,1646.0,1497.0,1899.0,1,A1,570000,2023-08-23
2,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3035,52,NaN,A1,"461 EAST 178TH STREET, NYC, BRONX",NaN,...,1.0,0.0,1.0,1782.0,1548.0,1899.0,1,A1,0,2023-04-14
3,BRONX,BATHGATE,01 ONE FAMILY DWELLINGS,1,3053,86,NaN,S0,"2364 WASHINGTON AVENUE, NYC, BRONX",NaN,...,1.0,2.0,3.0,1911.0,4080.0,1931.0,1,S0,0,2023-10-24
4,BRONX,BATHGATE,02 TWO FAMILY DWELLINGS,1,2904,22,NaN,B9,"454 EAST 172ND STREET, NYC, BRONX",NaN,...,2.0,0.0,2.0,1658.0,1428.0,1901.0,1,B9,350000,2023-06-26


In [16]:
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
73443,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,95,NaN,B9,"377 ENGLEWOOD AVENUE, NYC, STATEN ISLAND",NaN,...,2.0,0.0,2.0,3728.0,1960.0,1999.0,1,B9,790000,2022-12-23
73444,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,1,7359,1,NaN,B2,"104 GLADWIN STREET, NYC, STATEN ISLAND",NaN,...,2.0,0.0,2.0,4345.0,2200.0,2001.0,1,B2,0,2023-01-27
73445,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,1,7359,5,NaN,B2,"110 GLADWIN STREET, NYC, STATEN ISLAND",NaN,...,2.0,0.0,2.0,4372.0,2300.0,2001.0,1,B2,990000,2023-08-25
73449,STATEN ISLAND,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,6970,87,NaN,V0,"96 LENEVAR AVENUE, NYC, STATEN ISLAND",NaN,...,0.0,0.0,0.0,2644.0,0.0,2023.0,1,V0,1116638,2022-12-19
73450,STATEN ISLAND,WOODROW,22 STORE BUILDINGS,4,7144,510,NaN,K2,"3120 ARTHUR KILL ROAD, NYC, STATEN ISLAND",NaN,...,0.0,1.0,1.0,98116.0,7795.0,1977.0,4,K2,5600000,2022-12-19


## Geocoding

The address are now fixed. It is possible to geocode them and retrieve: latitude and longitude.

In [17]:
geocoder = GoogleV3(api_key=environ['GEOCODING_API_KEY'])

The following function will return a string where latitude and longitude of the input address are separated by a comma.

In [23]:
i = 0

In [19]:
def geocode_address(address):
    global i

    location = geocoder.geocode(address)

    if location is not None:
        response = f'{location.latitude},{location.longitude}'
    else:
        response = None

    print(f'{i}-{address}) {response}')
    i += 1
    
    return response 

## Dividing the datasets again

In [21]:
df_bronx = df[df['BOROUGH'] == 'BRONX']
df_brooklyn = df[df['BOROUGH'] == 'BROOKLYN']
df_manhattan = df[df['BOROUGH'] == 'MANHATTAN']
df_queens = df[df['BOROUGH'] == 'QUEENS']
df_statenisland = df[df['BOROUGH'] == 'STATEN ISLAND']

Since geocoding is a quiet long process, it's done separately for every borough and the result is stored in a separated (temporary) csv (one per borough), that will be merged into the final one. 

### Geocoding Bronx

In [ ]:
df_bronx.loc[:, 'TEMP_CORD'] = df_bronx.loc[:, 'ADDRESS'].map(geocode_address)

In [56]:
df_bronx.to_csv('../out/rollingsales_bronx.csv')

### Geocoding Staten Island

In [ ]:
df_statenisland['TEMP_CORD'] = df_statenisland.loc[:, 'ADDRESS'].map(geocode_address)

In [57]:
df_statenisland.to_csv('../out/rollingsales_statenisland.csv')

### Geocoding Manhattan

In [ ]:
df_manhattan.loc[:, 'TEMP_CORD'] = df_manhattan.loc[:, 'ADDRESS'].map(geocode_address)

In [58]:
df_manhattan.to_csv('../out/rollingsales_manhattan.csv')

### Geocoding Queens

In [ ]:
df_queens.loc[:, 'TEMP_CORD'] = df_queens.loc[:, 'ADDRESS'].map(geocode_address)

In [59]:
df_queens.to_csv('../out/rollingsales_queens.csv')

### Geocoding Brooklyn

In [ ]:
df_brooklyn.loc[:, 'TEMP_CORD'] = df_brooklyn.loc[:, 'ADDRESS'].map(geocode_address)

In [60]:
df_brooklyn.to_csv('../out/rollingsales_brooklyn.csv')

## Final cleaning

As in the beginning after geocoding process (that for security reasons happened separately), the datasets are read from their brand new csv files and concatenated into the final DataFrame.

In [61]:
df_bronx = pd.read_csv('../out/rollingsales_bronx.csv')
df_brooklyn = pd.read_csv('../out/rollingsales_brooklyn.csv')
df_manhattan = pd.read_csv('../out/rollingsales_manhattan.csv')
df_queens = pd.read_csv('../out/rollingsales_queens.csv')
df_statenisland = pd.read_csv('../out/rollingsales_statenisland.csv')

In [62]:
df = pd.concat(objs=[df_bronx, df_brooklyn, df_manhattan, df_queens, df_statenisland], axis='index').reset_index()

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72532 entries, 0 to 72531
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           72532 non-null  int64  
 1   Unnamed: 0.1                    72532 non-null  int64  
 2   Unnamed: 0                      72532 non-null  int64  
 3   BOROUGH                         72532 non-null  object 
 4   NEIGHBORHOOD                    72532 non-null  object 
 5   BUILDING CLASS CATEGORY         72532 non-null  object 
 6   TAX CLASS AT PRESENT            72467 non-null  object 
 7   BLOCK                           72532 non-null  int64  
 8   LOT                             72532 non-null  int64  
 9   EASEMENT                        0 non-null      float64
 10  BUILDING CLASS AT PRESENT       72467 non-null  object 
 11  ADDRESS                         72532 non-null  object 
 12  APARTMENT NUMBER                

In [64]:
df = df.drop(columns={'index', 'Unnamed: 0'})

Let's delete rows with a nan coordinate.

In [65]:
na_coordinates = df[df['TEMP_CORD'].isna()]
na_coordinates

,Unnamed: 0.1,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,TEMP_CORD
58551,15528,QUEENS,MIDDLE VILLAGE,01 ONE FAMILY DWELLINGS,1,2839,30,NaN,A1,"60-50 69TH LANE, NYC, QUEENS",...,0.0,1.0,3400.0,1188.0,1940.0,1,A1,0,2022-12-30,NaN


In [66]:
df = df.drop(index=na_coordinates.index)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72531 entries, 0 to 72531
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         72531 non-null  object 
 1   NEIGHBORHOOD                    72531 non-null  object 
 2   BUILDING CLASS CATEGORY         72531 non-null  object 
 3   TAX CLASS AT PRESENT            72466 non-null  object 
 4   BLOCK                           72531 non-null  int64  
 5   LOT                             72531 non-null  int64  
 6   EASEMENT                        0 non-null      float64
 7   BUILDING CLASS AT PRESENT       72466 non-null  object 
 8   ADDRESS                         72531 non-null  object 
 9   APARTMENT NUMBER                17439 non-null  object 
 10  ZIP CODE                        72531 non-null  float64
 11  RESIDENTIAL UNITS               54638 non-null  float64
 12  COMMERCIAL UNITS                41633

Extract latitude and longitude.

In [72]:
df['LATITUDE'] = df['TEMP_CORD'].str.split(r',').str[0]
df['LONGITUDE'] = df['TEMP_CORD'].str.split(r',').str[1]

Takes only necessary features.

In [74]:
df_out = df.loc[:, ['TAX CLASS AT PRESENT', 'NEIGHBORHOOD', 
                'LAND SQUARE FEET', 'SALE PRICE', 'YEAR BUILT', 
                'ADDRESS', 'LATITUDE', 'LONGITUDE']]

In [75]:
df_out = df_out.rename(columns={
    'TAX CLASS AT PRESENT': 'tax_clas',
    'NEIGHBORHOOD': 'neighborhood',
    'LAND SQUARE FEET': 'sqft', 
    'SALE PRICE': 'price',
    'YEAR BUILT': 'construction_year',
    'ADDRESS': 'address',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude'
})

In [76]:
df_out

,tax_clas,neighborhood,sqft,price,construction_year,address,latitude,longitude
0,1,BATHGATE,1646.0,215000,1899.0,"4453 PARK AVENUE, NYC, BRONX",40.8539858,-73.8962069
1,1,BATHGATE,1646.0,570000,1899.0,"4453 PARK AVENUE, NYC, BRONX",40.8539858,-73.8962069
2,1,BATHGATE,1782.0,0,1899.0,"461 EAST 178TH STREET, NYC, BRONX",40.84862469999999,-73.89768889999999
3,1,BATHGATE,1911.0,0,1931.0,"2364 WASHINGTON AVENUE, NYC, BRONX",40.8566642,-73.8917216
4,1,BATHGATE,1658.0,350000,1901.0,"454 EAST 172ND STREET, NYC, BRONX",40.8402829,-73.9025348
...,...,...,...,...,...,...,...,...
72527,1,WOODROW,3728.0,790000,1999.0,"377 ENGLEWOOD AVENUE, NYC, STATEN ISLAND",40.53231630000001,-74.22278109999999
72528,1,WOODROW,4345.0,0,2001.0,"104 GLADWIN STREET, NYC, STATEN ISLAND",40.531902,-74.2224952
72529,1,WOODROW,4372.0,990000,2001.0,"110 GLADWIN STREET, NYC, STATEN ISLAND",40.5316757,-74.22254099999999
72530,1B,WOODROW,2644.0,1116638,2023.0,"96 LENEVAR AVENUE, NYC, STATEN ISLAND",40.5388612,-74.20944899999999


In [77]:
df_out.to_csv('../out/rollingsales.csv')